<a href="https://colab.research.google.com/github/nichd36/Outpaint-Test/blob/main/OutpaintingControlNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Using cached diffusers-0.32.0.dev0-py3-none-any.whl


In [3]:
import random
import torch
import sympy
from PIL import Image, ImageOps
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel

In [4]:
controlnet = ControlNetModel.from_pretrained(
    "destitech/controlnet-inpaint-dreamer-sdxl", torch_dtype=torch.float16, variant="fp16"
)
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(
    "RunDiffusion/Juggernaut-XL-v9",
    torch_dtype=torch.float16,
    variant="fp16",
    controlnet=controlnet,
).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

...

unet/config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
prompt = "something cold"
# prompt = input("Kindly put in your prompt: ")

In [7]:
uploaded_image_path = input("Enter the path to your image file (jpg, png, jpeg): ")

Enter the path to your image file (jpg, png, jpeg): photo.jpg


In [8]:
print("Expand to:")
left = input("Expand left? (yes/no): ").strip().lower() == "yes"
right = input("Expand right? (yes/no): ").strip().lower() == "yes"
up = input("Expand up? (yes/no): ").strip().lower() == "yes"
down = input("Expand down? (yes/no): ").strip().lower() == "yes"

Expand to:
Expand left? (yes/no): yes
Expand right? (yes/no): yes
Expand up? (yes/no): no
Expand down? (yes/no): no


In [9]:
try:
    if uploaded_image_path:
        seed = random.randint(0, 2**32 - 1)

        image = Image.open(uploaded_image_path).convert("RGB")
        border_left = 500 if left else 0
        border_up = 500 if up else 0
        border_right = 500 if right else 0
        border_down = 500 if down else 0

        padded_image = ImageOps.expand(
            image, border=(border_left, border_up, border_right, border_down), fill=(255, 255, 255)
        )

        padded_image_path = "padded_image.jpg"
        padded_image.save(padded_image_path)
    else:
        print("No image uploaded.")
except Exception as e:
    print(f"Error processing image: {e}")

In [10]:
expanded_image = pipeline(
        prompt=prompt,
        height=padded_image.height,
        width=padded_image.width,
        guidance_scale=6.5,
        num_inference_steps=25,
        image=padded_image,
        controlnet_conditioning_scale=0.9,
        control_guidance_end=0.9,
    ).images[0]

expanded_image_path = "expanded_image.jpg"
expanded_image.save(expanded_image_path)

  0%|          | 0/25 [00:00<?, ?it/s]